Auto reload magics will dynamically load changes to the manual_review_classifier package for convenience during development [source](http://blog.comperiosearch.com/blog/2015/05/11/using-ipython-notebooks-and-pycharm-together/)

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%aimport manual_review_classifier

Populating the interactive namespace from numpy and matplotlib


In [2]:
import re, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn import preprocessing

In [4]:
import pickle

In [5]:
from manual_review_classifier.ReadCount import ReadCount

In [6]:
# pd.set_option('display.max_columns', 5000)
# pd.set_option('display.max_rows', 500)

### Traverse all samples and concatenate DFs

In [7]:
# Read in dataframe of sample information and variant calls
data = pd.read_pickle('../data/full_df.pkl')

# Select tumor normal pairs and groupby individual
g = data[data.tumor_normal_pair].groupby('individual_name')

counts = pd.DataFrame()
for name, group in g:
    
    print(name)
    individual_df = pd.DataFrame()
    normal_count=tumor_count=relapse_count=met_count=rna_count=1   
    for index, row in group[['build_id','tissue_type', 'sequencing_context', 'project', 'disease']].drop_duplicates().iterrows():
        if row['tissue_type'] == 'normal':
            column_name = 'normal_' + str(normal_count)
            normal_count += 1
        elif row['sequencing_context'] == 'rna seq':
            column_name = 'tumor_rna_' + str(rna_count)
            rna_count += 1
        elif row['tissue_type'] == 'tumor':
            column_name = 'tumor_' + str(tumor_count)
            tumor_count += 1
        elif row['tissue_type'] == 'relapse':
            column_name = 'relapse_' + str(relapse_count)
            relapse_count += 1
        elif row['tissue_type'] == 'met':
            column_name = 'met_' + str(met_count)
            met_count += 1
        print('\t'+column_name)
        print('\t'+row['build_id'],row['tissue_type'],  row['sequencing_context'], row['project'], row['disease'])
        count_fp = '../data/bam-read-counts/normal/{0}/{1}.counts'.format(name,row['build_id'])
#         print('\t\t'+count_fp)
        rc = ReadCount(count_fp)
        bed_fp = '../data/bam-read-counts/normal/{0}/{0}_full.bed'.format(name)
        temp_df = rc.compute_variant_metrics(bed_fp, column_name)
# TODO: Reinvestigate how to incorporate project, when you figure out what the primary is for each sample and what reviewers performed each project. Includinge this now results in merge errors.
#         temp_df['project'] = row['project']
        temp_df['disease'] = row['disease']
        if len(individual_df)==0:
            individual_df=temp_df
#             if name == 'H_KA-115225':
#                 if column_name == 'normal_1':
#                     save_n1 = temp_df
        else:
            if len(individual_df) != len(temp_df):
                bed_df = pd.read_csv(bed_fp, names=['chromosome','start','stop','ref','var','call'])
                raise ValueError('Dataframes cannot be merged. They are differing lengths.')
            individual_df = pd.merge(individual_df, temp_df, on=['chromosome', 'start', 'stop', 'ref', 'var', 'call', 'disease'])
#             if name == 'H_KA-115225':
#                 if column_name == 'tumor_1':
#                     save_t1 = temp_df
#                 elif column_name == 'normal_2':
#                     save_n2 = temp_df
            if len(individual_df) == 0:
                break
                print('this should not happen')
    if len(individual_df) == 0:
        break
        print('this should not happen')
    if any(individual_df.tumor_1_VAF.isnull()):
        break
    individual_df['individual_name']=name
    counts = pd.concat([counts, individual_df], ignore_index=True)


H_BS-312340G
	normal_1
	4ed71ccd17df455dae7200fb136b8ce2 normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	6c19a2eb9ef3448491c7f6ff7c05e67d tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_BS-807615G
	normal_1
	4b0aba5fed324014bf82eb0cb880342f normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	b0d8896570b64b3b97b49d49297dfe34 tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_GM-113971
	normal_1
	f54e721735334d08b954e1ec59005788 normal capture allegra/AML_Post-Transplant_Relapse AML
	tumor_1
	bb45156f99014815993e189982c158db tumor capture allegra/AML_Post-Transplant_Relapse AML
	met_1
	755cecc25a5a4aa9b92a01cb71f43ca1 met capture allegra/AML_Post-Transplant_Relapse AML
H_GM-456892
	normal_1
	b3d14d1ba54e4b60a0a48c894929c373 normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	cb6fae2054464abbb33b3cda8757f809 tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_GM-545259
	normal_1
	a5a75a140f9143559705b40b72378fa3 normal capture gue_su/SMR_06-09-rAML-review1 AML
	tum

H_KA-296361
	normal_1
	1c1a34b195074b93acc6cdcc2cab5305 normal capture allegra/AML_Post-Transplant_Relapse AML
	tumor_1
	10f20c44ceae4a2894c24accec0487be tumor capture allegra/AML_Post-Transplant_Relapse AML
	relapse_1
	7aa53411e46045ffa3a475eadb5bc32d relapse capture allegra/AML_Post-Transplant_Relapse AML
H_KA-303642
	normal_1
	49195700056d4af989beed46e40315ba normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	0169f26f7bd84fad8f47e137e8fe45cf tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_KA-303818
	normal_1
	19288b6154bc44d0a568ab48da5f780b normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	41fd90d1250b4e7bb2fc9efbc3a20bfa tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_KA-306905
	normal_1
	5ea1129ea176479181b589b5af3ad94a normal WGS lee/ALL1 AML
	tumor_1
	139036895454437da3b4a150bc045a50 tumor WGS lee/ALL1 AML
	tumor_2
	3bb8ec447b214a4b92874b7787805361 tumor WGS lee/ALL1 AML
	relapse_1
	3cb9ae27fa3d482bb672b0b31519bc47 relapse WGS lee/ALL1 AML
	relapse_2
	ce67f

	fbd173870e544a899e38fba457d89111 normal capture allegra/AML_Post-Transplant_Relapse AML
	tumor_1
	f5d875f5f8304817832813fb906cc32e tumor capture allegra/AML_Post-Transplant_Relapse AML
	relapse_1
	6d32a1e77fa0424db8a5e94e9c5f6740 relapse capture allegra/AML_Post-Transplant_Relapse AML
H_KA-633734G
	normal_1
	1bda54ecb5bb4235b8f9239767351fa0 normal capture allegra/AML_Post-Transplant_Relapse AML
	relapse_1
	df081d2f64d5423491863bc73b4bc815 relapse capture allegra/AML_Post-Transplant_Relapse AML
	tumor_1
	a9d525327edc4199b36661dbf5dd2d0e tumor capture allegra/AML_Post-Transplant_Relapse AML
H_KA-635258
	normal_1
	545dc216a1824c0eb8a9ca8498db1446 normal capture gue_su/SMR_06-09-rAML-review1 AML
	tumor_1
	870f62c2f0184550ac55b46a7461c0ac tumor capture gue_su/SMR_06-09-rAML-review1 AML
H_KA-654840
	normal_1
	140ff55eebb5418d8c7ff3ccb4d34c25 normal capture allegra/AML_Decitabine AML
	tumor_1
	5f953792802d41f98cc20e225cdf775c tumor capture allegra/AML_Decitabine AML
H_KA-688691
	normal_1
	8f

H_LB-01-13
	tumor_1
	57122a596c584e398a6ca75aa1f80bd5 tumor capture GTB11 AML
	normal_1
	0b63e502d6cc4b8c831fea5fd494093f normal capture GTB11 AML
H_LB-02-21
	tumor_1
	29dff05d19b24d299d908fb4790806a1 tumor capture GTB11 AML
	normal_1
	2a0d06f9ae794c74ad2f2e27455f17ab normal capture GTB11 AML
H_LB-02-28
	tumor_1
	bdb5c9ea82264688a89ba54dbe388012 tumor capture GTB11 AML
	normal_1
	43c43c46dbe54c9fa2d221a0f05cc0be normal capture GTB11 AML
H_LB-02-30
	tumor_1
	175c83c972a643c991071cef11844d77 tumor capture GTB11 AML
	normal_1
	d99f1a5765394bdc9b693f6deb375e07 normal capture GTB11 AML
H_LB-02-75
	tumor_1
	09def20e2b76415b9971911b73a20aa0 tumor capture GTB11 AML
	normal_1
	715aeae97b3c4073b56cdf4539f004d3 normal capture GTB11 AML
H_LB-02-77
	tumor_1
	888f290b69e645b2a39d762b28252307 tumor capture GTB11 AML
	normal_1
	7ac7f83084e94716bfa5d8c267b49bd0 normal capture GTB11 AML
H_LB-02-86
	tumor_1
	2602cb8be1864679acfe5514c934a74c tumor capture GTB11 AML
	normal_1
	20ad462da1d84a5a8aea9516b379f

H_LB-747967
	tumor_1
	3b864f26dc824ca698022c871b06cc3c tumor capture GTB11 AML
	normal_1
	7e1afbdcd0be4e6e8c00d5bd1ab087f2 normal capture GTB11 AML
H_LB-753222
	tumor_1
	924b96cde7f0461e990992faff06f928 tumor capture GTB11 AML
	normal_1
	578549fd9a4540b3a485d71a091e56c2 normal capture GTB11 AML
H_LB-783437
	tumor_1
	57b54c3941144ba488abc5fd5dd29848 tumor capture GTB11 AML
	normal_1
	93acbd33182a4faaa948962b8ab206c7 normal capture GTB11 AML
H_LB-785104
	tumor_1
	62fc91a581234b07b396daac207622e1 tumor capture GTB11 AML
	normal_1
	df8cb04d53c84202b624906420de9e71 normal capture GTB11 AML
H_LB-791334
	tumor_1
	69d3f571b7bd457c9ce2c2ffae2b1bd4 tumor capture GTB11 AML
	normal_1
	0eb1abef52224cd18d525f641b4fd9e8 normal capture GTB11 AML
H_LB-804059
	tumor_1
	fb58a32702544dc1bb4ba14453d11f62 tumor capture GTB11 AML
	normal_1
	0c85f8ebd8364422a0b8fa5571353c2c normal capture GTB11 AML
H_LB-809605
	tumor_1
	f857cfb0c32046c3887efc6be4bb678e tumor capture GTB11 AML
	normal_1
	3029f22b3bd24fe58b7c92

...

H_OM-16676
	tumor_1
	4b02c6e3972b43b88126080e97735fd5 tumor WGS traztuzamab breast
	normal_1
	eab0e89bc67a440fa9024a84696fd849 normal WGS traztuzamab breast
	tumor_2
	0f1f8b3ba3ab41598278f4c40a4e8e93 tumor exome traztuzamab breast
	normal_2
	8b2837ff60c046ecb9ede3c3128fc345 normal exome traztuzamab breast
	tumor_rna_1
	a436e2f516f04e439d18f5a33a87d759 tumor rna seq traztuzamab breast
H_OM-16693
	tumor_1
	501926aae4834332969914a994b0272f tumor WGS traztuzamab breast
	normal_1
	5ed846df6af64dafa4790bf873558852 normal WGS traztuzamab breast
	tumor_2
	769a44a7012f461abf764a810686dad4 tumor exome traztuzamab breast
	normal_2
	befbcb9ecce044498233ae7ab1e87225 normal exome traztuzamab breast
	tumor_rna_1
	0ec50bda9b734d34884b317d059ec2b3 tumor rna seq traztuzamab breast
H_OM-16708
	tumor_1
	256541f355b24dd8b225f63c98a9b790 tumor WGS traztuzamab breast
	normal_1
	6dac4fbeb8eb4fe89ed3a923bc3302d9 normal WGS traztuzamab breast
	tumor_2
	64419e2a482e426598db0953ea218626 tumor exome traztuzamab br

	tumor_rna_1
	dff0c715d7b043c6ab22749ae25efc60 tumor rna seq traztuzamab breast
H_OM-6198
	tumor_1
	d285659c1d2949b4ad0238404efb9a34 tumor WGS traztuzamab breast
	normal_1
	93334b70c2b54a2187e8b99188d2db87 normal WGS traztuzamab breast
	tumor_2
	f9a7993b19434d448aec3c11f1c1fd9d tumor exome traztuzamab breast
	normal_2
	b2ef7489b63e498781af680b6e7d3071 normal exome traztuzamab breast
	tumor_rna_1
	f97b0056b32d4c268420c48e4dd03ea6 tumor rna seq traztuzamab breast
H_OM-7801
	tumor_1
	165f6ea7710c4139873752c89319f1a2 tumor WGS traztuzamab breast
	normal_1
	815a46a4eae24ce0b5ccc226a9c7056a normal WGS traztuzamab breast
	tumor_2
	51b3303af9fb469cbdc9f6e925b2b6be tumor exome traztuzamab breast
	normal_2
	535292d82c8c42418ef7dc6440c4c85d normal exome traztuzamab breast
	tumor_rna_1
	3701bfed6c9a4156975c029ce4c5ec56 tumor rna seq traztuzamab breast
H_OM-8085
	tumor_1
	0f5fd0861bc24911a07e3a9f0b88a000 tumor WGS traztuzamab breast
	normal_1
	db687395e2d54716800342ecbe1fd8f5 normal WGS traztuzamab

	80dd8155d2364a92b71eb9c4057ee3c2 tumor exome allegra/AML_Decitabine AML
H_OO-10DD-1016
	normal_1
	d9622964cc6648a798b8137f97eaa2eb normal exome allegra/AML_Decitabine AML
	tumor_1
	a12e0253fe254b409b782d7fb4d5c34e tumor exome allegra/AML_Decitabine AML
	tumor_2
	8f98a06eac0849d5932d6fa9e902910e tumor exome allegra/AML_Decitabine AML
	tumor_3
	51f16ac2e3614b71a8a7de9826d8ed68 tumor exome allegra/AML_Decitabine AML
	tumor_4
	c12eecb18c374d409187d6843ff94396 tumor exome allegra/AML_Decitabine AML
H_OO-10DD-1017
	normal_1
	340b1b61b49643ad9adf7e5967fdd2a8 normal exome allegra/AML_Decitabine AML
	tumor_1
	7cbdfed6b8144407a8f5a20176ad4e42 tumor exome allegra/AML_Decitabine AML
	tumor_2
	83ebc0149fe44e1db864ea87fedb4814 tumor exome allegra/AML_Decitabine AML
	tumor_3
	1760ed37141f44b9b3eb4b0c10a76573 tumor exome allegra/AML_Decitabine AML
	tumor_4
	6ae26f359c6e4a11a8f7b44a7795de74 tumor exome allegra/AML_Decitabine AML
	tumor_5
	25ae1ae6cb4b44dd873173d230a68158 tumor exome allegra/AML_Decita

H_RO-WU002
	tumor_1
	f07a093f26b24d6b82e79539fbe3fde9 tumor capture lee/BKM120 breast
	normal_1
	0f6fa5ba0d9744f0a0b63ceb27eebc61 normal capture lee/BKM120 breast
H_RO-WU004
	tumor_1
	c22485824fdd472dbc07553be085521d tumor capture lee/BKM120 breast
	normal_1
	aa100808ce0c4065b2c3107d7452b20b normal capture lee/BKM120 breast
H_RO-WU005
	tumor_1
	5680e130bc174d909402328c42eb896e tumor capture lee/BKM120 breast
	normal_1
	5aaca41fc2964e939eec5743a3decf5a normal capture lee/BKM120 breast
H_RO-WU008
	tumor_1
	e3bc403fe69c4643a142ae0df4aca890 tumor capture lee/BKM120 breast
	normal_1
	227d39980b6547f7973b2bec0a8ad5e0 normal capture lee/BKM120 breast
H_RO-WU011
	tumor_1
	bdd396240afd4ac1be9b2a86390e6cb5 tumor capture lee/BKM120 breast
	normal_1
	34236c2995a24208a7a6aacec07adf70 normal capture lee/BKM120 breast
H_RO-WU012
	tumor_1
	84325bc2ee3d41a28014c6aedd04e8d1 tumor capture lee/BKM120 breast
	normal_1
	5d8dbd0997ea44ff8e192a244cd70c97 normal capture lee/BKM120 breast
H_RO-WU014
	tumor_1
	2

	tumor_2
	789ac82fcc444c8a95347f2a346702d4 tumor exome lee/DLBCL lymphoma
H_UI-08-0075-535
	tumor_1
	59e5b92aaf2041929575b0bc769b45d1 tumor exome GTB19 AML
	normal_1
	ed9027fa5f654934be75da80b90611a8 normal exome GTB19 AML
	tumor_rna_1
	c7d44f470adf4c399af9d096d0408c9a tumor rna seq GTB19 AML


### Filter and normalize training data

In [8]:
training_data = counts[['call', 'chromosome',  'start', 'stop', 'ref', 'var', 'disease', 'individual_name', 'normal_1_VAF', 'normal_1_depth', 'normal_1_other_bases_count', 'normal_1_ref_avg_basequality', 'normal_1_ref_avg_clipped_length', 'normal_1_ref_avg_distance_to_effective_3p_end', 'normal_1_ref_avg_distance_to_q2_start_in_q2_reads', 'normal_1_ref_avg_mapping_quality', 'normal_1_ref_avg_num_mismaches_as_fraction', 'normal_1_ref_avg_pos_as_fraction', 'normal_1_ref_avg_se_mapping_quality', 'normal_1_ref_avg_sum_mismatch_qualities', 'normal_1_ref_count', 'normal_1_ref_num_minus_strand', 'normal_1_ref_num_plus_strand', 'normal_1_ref_num_q2_containing_reads', 'normal_1_var_avg_basequality', 'normal_1_var_avg_clipped_length', 'normal_1_var_avg_distance_to_effective_3p_end', 'normal_1_var_avg_distance_to_q2_start_in_q2_reads', 'normal_1_var_avg_mapping_quality', 'normal_1_var_avg_num_mismaches_as_fraction', 'normal_1_var_avg_pos_as_fraction', 'normal_1_var_avg_se_mapping_quality', 'normal_1_var_avg_sum_mismatch_qualities', 'normal_1_var_count', 'normal_1_var_num_minus_strand', 'normal_1_var_num_plus_strand', 'normal_1_var_num_q2_containing_reads', 'tumor_1_VAF', 'tumor_1_depth', 'tumor_1_other_bases_count', 'tumor_1_ref_avg_basequality', 'tumor_1_ref_avg_clipped_length', 'tumor_1_ref_avg_distance_to_effective_3p_end', 'tumor_1_ref_avg_distance_to_q2_start_in_q2_reads', 'tumor_1_ref_avg_mapping_quality', 'tumor_1_ref_avg_num_mismaches_as_fraction', 'tumor_1_ref_avg_pos_as_fraction', 'tumor_1_ref_avg_se_mapping_quality', 'tumor_1_ref_avg_sum_mismatch_qualities', 'tumor_1_ref_count', 'tumor_1_ref_num_minus_strand', 'tumor_1_ref_num_plus_strand', 'tumor_1_ref_num_q2_containing_reads', 'tumor_1_var_avg_basequality', 'tumor_1_var_avg_clipped_length', 'tumor_1_var_avg_distance_to_effective_3p_end', 'tumor_1_var_avg_distance_to_q2_start_in_q2_reads', 'tumor_1_var_avg_mapping_quality', 'tumor_1_var_avg_num_mismaches_as_fraction', 'tumor_1_var_avg_pos_as_fraction', 'tumor_1_var_avg_se_mapping_quality', 'tumor_1_var_avg_sum_mismatch_qualities', 'tumor_1_var_count', 'tumor_1_var_num_minus_strand', 'tumor_1_var_num_plus_strand', 'tumor_1_var_num_q2_containing_reads']]

In [9]:
# Sanity check: Ensure all columns have data (expect 0)
len(training_data[training_data.isnull().any(axis=1)])

0

In [10]:
len(training_data)

37192

#### How many calls are there of each type?

In [11]:
training_data.groupby('call').size()

call
a    13745
f     1600
g     3946
s    17781
v      120
dtype: int64

'v' calls seem to be a bit of an outlier, additionally the call does not have a clear definition in the manual review guidelines. It it likely they are really messy, because of this I will remove them from future analysis.

In [12]:
training_data = training_data[training_data.call != 'v']

In [13]:
training_data.groupby('call').size()

call
a    13745
f     1600
g     3946
s    17781
dtype: int64

In [14]:
training_data.index = training_data.chromosome + ':' + training_data.start.map(str)+ '-' + training_data.stop.map(str) + training_data['ref'] + '>' + training_data['var']

In [15]:
# Remove genomic location information for now. 
# TODO: include important genomic features (ie predicticted function, region complexity, splicing information, etc)
# Remove individual_name
# ???: Could the individual_name be helpful for the classifier
training_data.drop(['chromosome', 'start', 'stop', 'ref', 'var', 'individual_name'], axis=1, inplace=True)

In [16]:
# Transform disease column in to boolean columns
training_data = pd.concat([training_data, pd.get_dummies(training_data['disease'], prefix='disease')], axis=1)
training_data.drop('disease', axis=1, inplace=True)

#### Normalize bam-readcount metrics

In [17]:
to_normalize = training_data[['normal_1_VAF', 'normal_1_depth', 'normal_1_other_bases_count',
       'normal_1_ref_avg_basequality', 'normal_1_ref_avg_clipped_length',
       'normal_1_ref_avg_distance_to_effective_3p_end',
       'normal_1_ref_avg_distance_to_q2_start_in_q2_reads',
       'normal_1_ref_avg_mapping_quality',
       'normal_1_ref_avg_num_mismaches_as_fraction',
       'normal_1_ref_avg_pos_as_fraction',
       'normal_1_ref_avg_se_mapping_quality',
       'normal_1_ref_avg_sum_mismatch_qualities', 'normal_1_ref_count',
       'normal_1_ref_num_minus_strand', 'normal_1_ref_num_plus_strand',
       'normal_1_ref_num_q2_containing_reads', 'normal_1_var_avg_basequality',
       'normal_1_var_avg_clipped_length',
       'normal_1_var_avg_distance_to_effective_3p_end',
       'normal_1_var_avg_distance_to_q2_start_in_q2_reads',
       'normal_1_var_avg_mapping_quality',
       'normal_1_var_avg_num_mismaches_as_fraction',
       'normal_1_var_avg_pos_as_fraction',
       'normal_1_var_avg_se_mapping_quality',
       'normal_1_var_avg_sum_mismatch_qualities', 'normal_1_var_count',
       'normal_1_var_num_minus_strand', 'normal_1_var_num_plus_strand',
       'normal_1_var_num_q2_containing_reads', 'tumor_1_VAF', 'tumor_1_depth',
       'tumor_1_other_bases_count', 'tumor_1_ref_avg_basequality',
       'tumor_1_ref_avg_clipped_length',
       'tumor_1_ref_avg_distance_to_effective_3p_end',
       'tumor_1_ref_avg_distance_to_q2_start_in_q2_reads',
       'tumor_1_ref_avg_mapping_quality',
       'tumor_1_ref_avg_num_mismaches_as_fraction',
       'tumor_1_ref_avg_pos_as_fraction', 'tumor_1_ref_avg_se_mapping_quality',
       'tumor_1_ref_avg_sum_mismatch_qualities', 'tumor_1_ref_count',
       'tumor_1_ref_num_minus_strand', 'tumor_1_ref_num_plus_strand',
       'tumor_1_ref_num_q2_containing_reads', 'tumor_1_var_avg_basequality',
       'tumor_1_var_avg_clipped_length',
       'tumor_1_var_avg_distance_to_effective_3p_end',
       'tumor_1_var_avg_distance_to_q2_start_in_q2_reads',
       'tumor_1_var_avg_mapping_quality',
       'tumor_1_var_avg_num_mismaches_as_fraction',
       'tumor_1_var_avg_pos_as_fraction', 'tumor_1_var_avg_se_mapping_quality',
       'tumor_1_var_avg_sum_mismatch_qualities', 'tumor_1_var_count',
       'tumor_1_var_num_minus_strand', 'tumor_1_var_num_plus_strand',
       'tumor_1_var_num_q2_containing_reads']]

In [18]:
# Source http://stackoverflow.com/a/26415620
x = to_normalize.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled = pd.DataFrame(x_scaled, index=training_data.index, columns=['normal_1_VAF', 'normal_1_depth', 'normal_1_other_bases_count',
       'normal_1_ref_avg_basequality', 'normal_1_ref_avg_clipped_length',
       'normal_1_ref_avg_distance_to_effective_3p_end',
       'normal_1_ref_avg_distance_to_q2_start_in_q2_reads',
       'normal_1_ref_avg_mapping_quality',
       'normal_1_ref_avg_num_mismaches_as_fraction',
       'normal_1_ref_avg_pos_as_fraction',
       'normal_1_ref_avg_se_mapping_quality',
       'normal_1_ref_avg_sum_mismatch_qualities', 'normal_1_ref_count',
       'normal_1_ref_num_minus_strand', 'normal_1_ref_num_plus_strand',
       'normal_1_ref_num_q2_containing_reads', 'normal_1_var_avg_basequality',
       'normal_1_var_avg_clipped_length',
       'normal_1_var_avg_distance_to_effective_3p_end',
       'normal_1_var_avg_distance_to_q2_start_in_q2_reads',
       'normal_1_var_avg_mapping_quality',
       'normal_1_var_avg_num_mismaches_as_fraction',
       'normal_1_var_avg_pos_as_fraction',
       'normal_1_var_avg_se_mapping_quality',
       'normal_1_var_avg_sum_mismatch_qualities', 'normal_1_var_count',
       'normal_1_var_num_minus_strand', 'normal_1_var_num_plus_strand',
       'normal_1_var_num_q2_containing_reads', 'tumor_1_VAF', 'tumor_1_depth',
       'tumor_1_other_bases_count', 'tumor_1_ref_avg_basequality',
       'tumor_1_ref_avg_clipped_length',
       'tumor_1_ref_avg_distance_to_effective_3p_end',
       'tumor_1_ref_avg_distance_to_q2_start_in_q2_reads',
       'tumor_1_ref_avg_mapping_quality',
       'tumor_1_ref_avg_num_mismaches_as_fraction',
       'tumor_1_ref_avg_pos_as_fraction', 'tumor_1_ref_avg_se_mapping_quality',
       'tumor_1_ref_avg_sum_mismatch_qualities', 'tumor_1_ref_count',
       'tumor_1_ref_num_minus_strand', 'tumor_1_ref_num_plus_strand',
       'tumor_1_ref_num_q2_containing_reads', 'tumor_1_var_avg_basequality',
       'tumor_1_var_avg_clipped_length',
       'tumor_1_var_avg_distance_to_effective_3p_end',
       'tumor_1_var_avg_distance_to_q2_start_in_q2_reads',
       'tumor_1_var_avg_mapping_quality',
       'tumor_1_var_avg_num_mismaches_as_fraction',
       'tumor_1_var_avg_pos_as_fraction', 'tumor_1_var_avg_se_mapping_quality',
       'tumor_1_var_avg_sum_mismatch_qualities', 'tumor_1_var_count',
       'tumor_1_var_num_minus_strand', 'tumor_1_var_num_plus_strand',
       'tumor_1_var_num_q2_containing_reads'])

In [19]:
training_data = pd.concat([training_data[['call', 'disease_AML', 'disease_GST',
       'disease_MPNST', 'disease_SCLC', 'disease_breast', 'disease_colorectal',
       'disease_glioblastoma', 'disease_lymphoma', 'disease_melanoma']], scaled], axis=1)

In [20]:
training_data.to_pickle('../data/training_data.pkl')